In [2]:
import pypsa
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
from dictionaries import regions_dic, links_dic, res_potential_dic, demand_profile_dic
from pyproj import Geod
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)
plt.rc("figure", figsize=(50, 50))


In [7]:
regions_dic[10]["countries"]

['Belarus', 'Russia', 'Ukraine']

# Interconnectors

### Calculation of distances between nodes

In [2]:
# Create a Geod object for the WGS84 ellipsoid
geod = Geod(ellps="WGS84")

# Function that calculates the distance between two nodes
def calculate_distance(bus0, bus1):
    """Calculates the actual distance between two points on the earth.

    Args:
        node1 (int): Identifier of bus 0
        node2 (int): Identifier of bus 1

    Returns:
        float: Distance between the two nodes in km.
    """
    lon1, lat1 = regions_dic[bus0]["coordinates"]
    lon2, lat2 = regions_dic[bus1]["coordinates"]
    _, _, distance = geod.inv(lon1, lat1, lon2, lat2)
    return distance / 1000

# Compute the lengths of the interconnectors and store in dictionary
for link in links_dic:
    links_dic[link]["length"] = calculate_distance(links_dic[link]["bus0"][0], links_dic[link]["bus1"][0])


### Calculation of efficiency

In [3]:
def efficiency_link(P, V, rho, l):
    """Calculates the relative losses of each interconnector.

    Args:
        P (float): Power capacity of cable in MW
        V (float): Cable volatge in kV 
        rho (float): Resisitivity of cable in ohms/km
        l (float): length of cable in km

    Returns:
        float: Efficiency of the cable 
    """
    I = P / V * 1000 # Current in amperes (A)
    R = rho * l # Resistance in ohms 
    losses = I**2 * R * 1e-6 # total losses in MW
    efficiency =  1 - losses / P # Efficiency of the cable

    return efficiency

# Compute the efficiency of the interconnectors and store in dictionary
cable_capacity = 1200 # Capacity of cable in MW
cable_voltage = 1100 # Voltage of cable in kV
cable_resistivity = 0.01286 # Resistivity of the cable in ohms/km
converter_losses = 0.015 # rlative losses for the converter pair
for link in links_dic:
    links_dic[link]["efficiency"] = efficiency_link(cable_capacity, cable_voltage, cable_resistivity, links_dic[link]["length"]) - converter_losses

# PyPSA 

### Create the grid

In [20]:
# Initialise the network
globalgrid = pypsa.Network() 

# Loop over the dictionary to add the buses
for i in regions_dic:
    globalgrid.add(class_name = "Bus", name = regions_dic[i]["region"], v_nom=f'{cable_voltage}', carrier = "DC", v_mag_pu_set=1.0, v_mag_pu_min=0.95,v_mag_pu_max=1.05,x = regions_dic[i]["coordinates"][0] , y = regions_dic[i]["coordinates"][1]) 
    
# Loop over the dictionary to add the links
for link in links_dic:
    globalgrid.add(class_name="Link", name=links_dic[link]["name"], bus0=links_dic[link]["bus0"][1], bus1=links_dic[link]["bus1"][1], efficiency=links_dic[link]["efficiency"] , marginal_cost=0, p_min_pu=-1, p_nom_max= np.inf, p_nom_extendable=True)

# Loop over the dictionary to add the renewable profiles --> these are the renewable potential generated with Atlite
for node in res_potential_dic:
    globalgrid.add(class_name="Generator", name=f'Wind {res_potential_dic[node]["name"]}', type="Wind", bus=res_potential_dic[node]["name"], p_nom_extendable = True, p_nom_max=res_potential_dic[node]["wind_series"],p_nom_min=res_potential_dic[node]["wind_series"], marginal_cost = 0)
    globalgrid.add(class_name="Generator", name=f'PV {res_potential_dic[node]["name"]}', type="PV", bus=res_potential_dic[node]["name"], p_nom_extendable = True, p_nom_max=res_potential_dic[node]["pv_series"],p_nom_min=res_potential_dic[node]["pv_series"], marginal_cost = 0)
    globalgrid.add(class_name="Generator", name=f'Curtailment {res_potential_dic[node]["name"]}', type="PV", bus=res_potential_dic[node]["name"], p_nom_extendable = True, marginal_cost = 10000)

# Loop over the dictionary to add the demand profiles --> these are the demand profiles for each region
for node in demand_profile_dic:
    globalgrid.add(class_name="Load", name=f"{demand_profile_dic[node]["name"]}", bus=demand_profile_dic[node]["name"], p_set=demand_profile_dic[node]["demand_series"])
    

### Solve the optmisation problem

In [ ]:
# # Define a time range for an entire year with hourly resolution
# time_series = pd.date_range("2023-01-01 00:00", "2023-12-31 23:00", freq="h")

# # Set the snapshot time range for PyPSA
# globalgrid.set_snapshots(time_series)

# # Optimize the grid
# globalgrid.optimize(solver_name='glpk')

In [8]:
# globalgrid.plot(
#     title="Geo map of buses",
#     color_geomap=True,
#     bus_sizes=2,
#     bus_colors="red",
#     link_widths=5,
#     link_colors="blue"
# )